In [1]:
import numpy as np
import tensorflow as tf
import dpu_efn
from lite import efficientnet_lite_builder
import efficientnet_builder
from pathlib import Path
import shutil
import numpy as np
import utils
import torch
from scipy import special
import queue

## Rebuild efficientnet lite 0 for ultra96v2 board usage

In [2]:
images = tf.placeholder(tf.float32, shape=(1, 224, 224, 3), name='input')

override_params = {}
override_params['relu_fn'] = tf.nn.relu6

blocks_args, global_params = efficientnet_lite_builder.get_model_params('efficientnet-lite0',override_params)

override_filters={9:557,
                  10:567,
                  11:588,
                  12:768,
                  13:768,
                  14:768,
                  15:365,}

model = dpu_efn.Model(blocks_args, global_params, override_filters)
outputs = model(images,training=False)
outputs = tf.identity(outputs, 'logits')

print('Model Build complete!')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model Build complete!


In [3]:
sess = tf.Session()

init = tf.global_variables_initializer()
sess.run(init)

# load pytorch model and fold bn into conv

In [4]:
ckpt = torch.load('./quant_best_72_8.pth',map_location=torch.device('cpu'))
torch_weights = ckpt.state_dict()
torch_tensor_names = list(torch_weights.keys())

# Assign the pytorch weights to built model

In [5]:
all_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

ti = 0

for idx,v in enumerate(all_variables):
    torch_value = torch_weights[torch_tensor_names[ti]].numpy()

    # fix conv2d and depthwise conv "CNHW" weight format in pytorch
    # in standard tf, the default weight format is "HWNC"
    print(v.name,torch_tensor_names[ti])

    if 'dense' in v.name and not 'bias' in v.name:
        torch_value = np.transpose(torch_value,(1,0))
    elif 'depthwise' in v.name and 'kernel' in v.name:
        torch_value = np.transpose(torch_value,(2,3,0,1))
    elif 'kernel' in v.name:
        torch_value = np.transpose(torch_value,(2,3,1,0))
    
    v.load(torch_value,sess)
    
    ti += 1

stem/conv2d/kernel:0 module._conv_stem.weight
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


stem/conv2d/bias:0 module._conv_stem.bias
blocks_0/depthwise_conv2d/depthwise_kernel:0 module._blocks.0._depthwise_conv.weight
blocks_0/depthwise_conv2d/bias:0 module._blocks.0._depthwise_conv.bias
blocks_0/conv2d/kernel:0 module._blocks.0._project_conv.weight
blocks_0/conv2d/bias:0 module._blocks.0._project_conv.bias
blocks_1/conv2d/kernel:0 module._blocks.1._expand_conv.weight
blocks_1/conv2d/bias:0 module._blocks.1._expand_conv.bias
blocks_1/depthwise_conv2d/depthwise_kernel:0 module._blocks.1._depthwise_conv.weight
blocks_1/depthwise_conv2d/bias:0 module._blocks.1._depthwise_conv.bias
blocks_1/conv2d_1/kernel:0 module._blocks.1._project_conv.weight
blocks_1/conv2d_1/bias:0 module._blocks.1._project_conv.bias
blocks_2/conv2d/kernel:0 module._blocks.2._expand_conv.weight
blocks_2/conv2d/bias:0 module._blocks.2._expand_conv.bias
blocks_2/depthwise_conv2d/depthwise_kernel:0 module._blocks.2._depthwise_conv.weight
blocks_2/depthwise_conv2d/bias:0 module._blocks.2._depthwise_conv.bias
bl

# Export Model

In [6]:
simple_save_dir='./efn-dpu-0-simple-save'
dirpath = Path(simple_save_dir)
if dirpath.exists() and dirpath.is_dir():
    shutil.rmtree(dirpath)

In [7]:
tf.saved_model.simple_save(sess,'./efn-dpu-0-simple-save',inputs={"input":images},outputs={'logits':outputs})

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.simple_save.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: ./efn-dpu-0-simple-save/saved_model.pb


INFO:tensorflow:SavedModel written to: ./efn-dpu-0-simple-save/saved_model.pb


In [8]:
!freeze_graph\
    --input_saved_model_dir './efn-dpu-0-simple-save/'\
    --input_binary true\
    --output_graph ./efn_dpu_0_frozen.pb\
    --output_node_names logits

2020-09-10 21:09:44.617357: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-09-10 21:09:44.632934: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fb1bd9654e0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2020-09-10 21:09:44.632952: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
W0910 21:09:44.633364 4566756800 deprecation.py:323] From /Users/maolin/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/tools/freeze_graph.py:161: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in 

In [9]:
!scp ./efn_dpu_0_frozen.pb a2:/media/nvme/ultra-dev/

efn_dpu_0_frozen.pb                           100%   14MB  83.2MB/s   00:00    


In [10]:
#!scp . a2:/media/nvme/DPU-PYNQ/host/efn-dpu-p3/deploy_model.pb /efn_dpu_0_deploy_model.pb